In [ ]:
void
kvminit()
{
  kernel_pagetable = (pagetable_t) kalloc();//Kvminit 首先分配一页物理内存来保存根页表页。
  memset(kernel_pagetable, 0, PGSIZE);

  // uart registers
  kvmmap(UART0, UART0, PGSIZE, PTE_R | PTE_W);

  // virtio mmio disk interface
  kvmmap(VIRTIO0, VIRTIO0, PGSIZE, PTE_R | PTE_W);

  // CLINT
  kvmmap(CLINT, CLINT, 0x10000, PTE_R | PTE_W);

  // PLIC
  kvmmap(PLIC, PLIC, 0x400000, PTE_R | PTE_W);

  // map kernel text executable and read-only.
  kvmmap(KERNBASE, KERNBASE, (uint64)etext-KERNBASE, PTE_R | PTE_X);
  // map kernel data and the physical RAM we'll make use of.
  kvmmap((uint64)etext, (uint64)etext, PHYSTOP-(uint64)etext, PTE_R | PTE_W);

  // map the trampoline for trap entry/exit to
  // the highest virtual address in the kernel.
  kvmmap(TRAMPOLINE, (uint64)trampoline, PGSIZE, PTE_R | PTE_X);
}

In [ ]:
#define PGROUNDUP(sz)  (((sz)+PGSIZE-1) & ~(PGSIZE-1))//将sz上对齐到页大小的整数倍
#define PGROUNDDOWN(a) (((a)) & ~(PGSIZE-1))//将sz下对齐到a的整数倍


Question:
#define PA2PTE(pa) ((((uint64)pa) >> 12) << 10)
#define PTE2PA(pte) (((pte) >> 10) << 12)分配一个页表时，需要调用PA2PTE将页表的真实物理地址转化为页表项PTE，页表项怎么根据PTE2PA得到真实的页表地址呢？
一个地址先使用PA2PTE右移12位再左移十位，在使用PTE2PA右移十位左移12位，那原来12位上的数据不是被0代替了吗？
A:任何有效的物理地址的最低 12 位在页表管理系统中始终是 0（因为它们表示页内偏移）。由于页的最小单位是 4KB，页的基地址必定是 4KB 对齐的。因此，物理地址的低 12 位在页表中不需要存储，也正因为如此，我们可以放心地忽略它们。

Question:
当 walk 下降页表级别时，会从 PTE中提取下一级页表的（物​​理）地址，然后使用该地址作为虚拟地址来获取下一级的 PTE，为什么不直接使用该地址作为物理地址来获取下一级的 PTE？
A：
1.虚拟地址简化内存访问的统一性,操作系统和 CPU 通常工作在虚拟内存模式下，所有的内存访问，甚至内核自身的内存访问，也依赖虚拟地址。为了简化设计，内核也通过虚拟地址访问页表中的物理内存，使得所有内存操作遵循统一的模式。
2.直接映射内核地址空间：内核通过直接映射（direct-mapped region）将物理内存的一部分映射到固定的虚拟地址空间。这种机制让内核可以直接通过虚拟地址访问物理内存中的页表，从而不需要每次通过物理地址进行访问。这意味着，当从 PTE 中提取到物理地址后，内核通过一个直接映射区域将该物理地址转换成虚拟地址进行访问。这让访问效率更高，也避免了手动管理物理地址的问题。
3.平台无关性:不同硬件平台的物理内存布局和管理可能有所不同。虚拟地址提供了一层抽象，使得内核代码在不同平台上保持一致，而不依赖具体的物理地址空间布局。
使用虚拟地址访问内存还可以让内核代码更加通用，便于跨平台移植，而不需要特定处理物理地址的访问逻辑。
4.缓存一致性与管理：
现代处理器使用多级缓存和 TLB（Translation Lookaside Buffer）来加速虚拟地址到物理地址的转换和内存访问。如果内核直接使用物理地址，会绕过虚拟内存管理机制，破坏缓存和 TLB 的一致性。使用虚拟地址可以确保这些机制有效工作，提升访问速度。尤其是在大规模多核系统中，直接使用物理地址访问可能导致复杂的缓存一致性问题。

Q：for(p = proc; p < &proc[NPROC]; p++) {
这个代码中，p是一个指针存储着地址，那么p++是地址+1还是地址加一个proc大小？
A：当你对一个指针进行递增操作（如 p++），编译器会根据该指针所指向的数据类型来决定递增多少字节。
对于指向 struct proc 类型的指针 p，p++ 实际上会让 p 指向下一个 struct proc 结构体的内存地址。
指针类型决定步长。

Q: 
struct proc *p;release(&p->lock);解释release(&p->lock)代码中为什么需要加&
A:
不加取地址符传递的话，则发送的将是锁定的值（通常是一个复制体），而不是其地址。这样release()无法操作实际的锁定，程序逻辑就无法正确工作。且C 语言没有引用传递，只有值传递，因此需要通过传递指针（地址）来操作实际的锁。

In [ ]:
Q：
代码：int
copyin_new(pagetable_t pagetable, char *dst, uint64 srcva, uint64 len)
{
  struct proc *p = myproc();

  if (srcva >= p->sz || srcva+len >= p->sz || srcva+len < srcva)
    return -1;
  memmove((void *) dst, (void *)srcva, len);
  stats.ncopyin++;   // XXX lock
  return 0;
}
解释：为什么在copyin_new()中需要第三个测试srcva + len < srcva：
给出srcva和len值的例子，这样的值将使前两个测试为假（即它们不会导致返回-1），但是第三个测试为真 （导致返回-1）。
A：
前两个条件 检查的是 srcva 和 srcva + len 是否在进程的合法地址空间内。
第三个条件 检查 srcva + len 是否由于整数溢出导致变为一个非法的较小地址，确保对内存操作的正确性和安全性。

In [ ]:
用户进程拥有自己的内存，内核掌握整个系统的物理内存管理。
为什么要创建进程的内核页表副本：
解决内核不能直接使用用户空间的虚拟地址这一问题。<br>
由于在标准的xv6实现中，内核页表和用户页表是分开的，也就是说用户页表包含的是用户进程的虚拟地址到物理地址的映射，用户进程运行时使用用户页表。
内核页表只包含内核空间的虚拟地址到物理地址的映射，内核代码运行时使用内核页表。每当内核需要访问用户进程的内存时（如处理系统调用或进行内存管理），它需要通过一些机制（如 copyin() 和 copyout()）来间接访问用户进程的内存。
改进方法：为每个进程创建一个内核页表副本，并在进程进入内核态时使用这个副本。<br>
这种设计能让内核直接访问用户地址空间，从而避免使用 copyin() 和 copyout() 之类的函数来访问用户空间数据。<br>
操作系统除了为该进程创建用户页表外，还会为该进程创建一个内核页表的副本。这个页表将包含以下两部分内容：<br>
用户地址空间的映射：包含用户空间的虚拟地址到物理地址的映射，确保内核在处理用户地址时能够直接访问。<br>
内核地址空间的映射：和标准内核页表一样，包含内核空间的虚拟地址到物理地址的映射，确保内核代码能正常执行。
在进入内核态时切换页表： 当进程从用户态切换到内核态时（例如通过系统调用、中断、异常等），内核会切换到该进程的内核页表副本。这样，内核执行时既能访问自己的内核空间，也能直接访问用户空间的虚拟地址。<br>
在上下文切换时切换页表： 当进程上下文切换时，操作系统不仅需要切换到新的进程的用户页表，还需要切换到该进程的内核页表副本，以确保内核在执行时使用正确的地址空间映射。<br>
保持内核和用户地址空间分离： 内核页表副本中的用户空间映射是只读的，这样即使内核能够访问用户空间地址，它也不能意外修改用户空间的数据。这可以通过在页表项中设置适当的权限位来实现。<br>
优点：<br>
简化内核与用户态的交互： 由于每个进程的内核页表副本包含了用户空间的映射，内核可以直接使用用户空间的虚拟地址，从而简化了内核代码.<br>
统一的地址空间： 在这种设计下，内核和用户进程实际上共享了同一个页表结构，这样就减少了页表切换带来的性能损失。<br>
提高性能： 这种方法可以减少用户态和内核态之间的开销，因为内核不需要频繁进行虚拟地址到物理地址的转换。直接访问用户空间的数据会更加高效。


In [ ]:
mknod 是一个系统调用，用来创建设备节点（文件系统中的特殊文件，通常用于访问硬件设备）。
r = Runner(save("xv6.out"))
@test(5, "count copyin")
def test_count():
    r.run_qemu(shell_script([
        'stats', 'stats'
    ]), timeout=150)
    matches = re.findall(r'^copyin: (\d+)', r.qemu.output, re.MULTILINE)
    assert_equal(len(matches), 2)
    assert_equal(int(matches[1]), int(matches[0]) + 28)
    matches = re.findall(r'^copyinstr: (\d+)', r.qemu.output, re.MULTILINE)
    assert_equal(len(matches), 2)
    assert_equal(int(matches[1]), int(matches[0]) + 3)